-----
** Ne pas faire un "execute all" : la dernière cellule est _très_ lourde. **

-----


# Introduction à TensorFlow

Ce code est basé sur des tutoriel à [tensorflow.org](https://www.tensorflow.org/).

Nous allons utiliser _softmax regression  pour MNIST.

La première chose à faire est de récupérer les données.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Exercice : explorer les données (mnist).  Qu'est-ce que vous trouvez, comprenez ?

## Softmax regression

Softmax est simplement une généralisation de la fonction logit (sigmoid) qui garantie qu'un vecteur finit entre 0 et 1.  Dit différemment, l'indice qu'une entrée $x$ est dans class $i$ est

$$\mbox{evidence}_i = \sum_j W_{i,j} x_j + b_i$$

avec

$$\mbox{softmax}(x) = \mbox{normalize}(e^x) \iff \mbox{softmax}(x)_i = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

Souvent on écrit simplement

$$y = \mbox{softmax}(Wx+b)$$

## Cross-entropy

$$H_{y'}(y) = -\sum_i y_i' \log(y_i)$$

## TensorFlow (I)

In [ ]:
import tensorflow as tf

# Ici None veut dire "de n'importe quelle longueur".
x = tf.placeholder(tf.float32, [None, 784])

# Nous allons apprendre W et b, donc on s'inquiète pas
# de leur valeurs maintenant.
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Notre modèle.
y = tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-entropy, cf. théorie de l'information.
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), 
                                              reduction_indices=[1]))
# En fait, ça manque de stabilité numérique, dans la vrai vie pensez
# à utiliser tf.nn.(sparse_)softmax_cross_entropy_with_logits().

# Optimisation, au choix.
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Initialiser toutes les variables.
init = tf.initialize_all_variables()

# Jusqu'ici nous n'avons rien fait.  Il ne s'agit que déclarations.
# Maintenant on déclenche le calcul.
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})


In [ ]:
# tf.argmax nous donne la valeur la plus importante sur une axe
# d'un tenseur.  Donc tf.argmax(y, 1) est la labelle que notre
# modèle trouve le plus probable pour chaque entrèe, et
# tf.argmax(y_, 1) est la labelle correcte.

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, 
                                    y_: mnist.test.labels}))

## TensorFlow (II)

La même chose mais une session interactive, ce qui nous permet d'être un peu plus lâche dans l'ordre d'opérations.

In [ ]:
import tensorflow as tf

sess2 = tf.InteractiveSession()

x2 = tf.placeholder(tf.float32, shape=[None, 784])
y_2 = tf.placeholder(tf.float32, shape=[None, 10])
W2 = tf.Variable(tf.zeros([784,10]))
b2 = tf.Variable(tf.zeros([10]))
sess2.run(tf.initialize_all_variables())

y2 = tf.matmul(x2, W2) + b2
cross_entropy2 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(y2, y_2))
train_step2 = tf.train.GradientDescentOptimizer(0.5).minimize(
    cross_entropy2)
for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step2.run(feed_dict={x2: batch[0], y_2: batch[1]})


In [ ]:
correct_prediction2 = tf.equal(tf.argmax(y2, 1), tf.argmax(y_2, 1))
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
print(accuracy2.eval(feed_dict={x2: mnist.test.images, 
                                y_2: mnist.test.labels}))

## TensorFlow (III)

Construisons un modèle plus soutenu : un multilayer convolutional network.

Nous avons vu des neurones de type rectified linear, $f(x) = max(0, x)$.  Un tel neurone s'appelle un ReLU.

On peut le rendre différentiable ainsi : $ f(x) = \ln\left(1+e^x\right)$.

Remarques :
* On leur donne un biais légèrement positif afin d'éviter des neurones "morts".
* On ajoute un peut de bruit pour briser symétrie ("symmetry breaking").


In [ ]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

x3 = tf.placeholder(tf.float32, shape=[None, 784])
y_3 = tf.placeholder(tf.float32, shape=[None, 10])

### Première couche

C'est une convolution de _stride_ 1 et _zero padded_, donc la sortie a la même taille que l'entrée. On suit la convolution du _max pooling_.

La convolution calculera 32 critères pour chaque carré (_patch_) de 5$\times$ 5.  Le tenseur de poids est de dimension $5\times 5\times 1\times 32$.  Les dimensions sont (2) _patch size_, (1) nombres de chenals d'entrée, et (1) le nombre de chenals de sortie.

Afin de l'appliquer, nous reformerons notre image ($28\times 28$) à un tenseur 4-D.  Le quatrième dimension est le nombre de couleurs dans l'image.

Finalement, nous passons par la convolution de l'image avec le poid, on ajoute le biais, passe par le ReLU et puis le max pool.

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x3, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Deuxième couche

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


### Troisième couche

*Densely connected layer.*  L'image est réduite à $7\times 7$.  Nous ajoutons une couche entièrement connectée avec 1024 neurones afin de traiter l'image entière.

Nous reformons le tenseur de la couche précédente ("pooling layer") dans une collection de vecteur, on multiplie pare la matrice de poids, ajoute un biais, et passe par un ReLU.

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout

Afin de réduire le surapprentissage, nous ajoutons un dropout avant la sortie finale.  Le placeholder représente la probabilité qu'un neuronne est gardé pendant la phase de dropout.  Ainsi on peut utiliser le dropout en apprentissage mais pas en classification.

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout

In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### Apprentissage et évaluation

Ici c'est presque la même chose qu'avec softmax.  Les différences principales :
* On remplace la descente du gradient avec ADAM.
* On ajoute `keep_prob` au `feed_dict`.
* On ajoute un message de logging toutes les 100 itéreation.

**Attention : le code suivant fait 20K itérations et risque de prendre un moment (approx. 30 minutes) !**

In [ ]:
sess3 = tf.Session()
# num_iterations = 20 * 1000
num_iterations = 800

cross_entropy3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        y_conv, y_3))
train_step3 = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy3)
correct_prediction3 = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_3,1))
accuracy3 = tf.reduce_mean(tf.cast(correct_prediction3, tf.float32))
sess3.run(tf.initialize_all_variables())
for i in range(num_iterations):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy3.eval(feed_dict={
            x3: batch[0], y_3: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step3.run(feed_dict={x3: batch[0], y_3: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
            x3: mnist.test.images, 
            y_3: mnist.test.labels, 
            keep_prob: 1.0}))